In [22]:
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg, brown
import random
from math import ceil, inf
import copy
from english_words import english_words_lower_set
from collections import defaultdict
from math import log10
from math import exp

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/thanhan/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [38]:
# General class for n-gram
class N_gram:
    def __init__(self, n=1):
        self.n = n
        self.prob_dict = {}
        self.min_prob = 1.0  # used when we get 0 prob
        self.dictionary = defaultdict(lambda: 1)
        self.longest_word_length = 0
        self.cnt = 0

    # Function to train the model using a given corpus
    def train(self, corpus):
        count_n = {}
        count_n_1 = {}

        for sentence in corpus:
            cur_context = ""
            for i in range(len(sentence)):
                if cur_context in count_n_1:
                    count_n_1[cur_context] += 1
                else:
                    count_n_1[cur_context] = 1
                if (sentence[i], cur_context) in count_n:
                    count_n[(sentence[i], cur_context)] += 1
                else:
                    count_n[(sentence[i], cur_context)] = 1
                # print((corpus[i],cur_context))
                self.dictionary[sentence[i]] += 1
                self.cnt += 1
                self.longest_word_length = max(
                    self.longest_word_length, len(sentence[i])
                )
                # if(sentence[i] in ['t','h','e','f']): print(sentence[i], sentence)
                cur_context += sentence[i]
                if i >= self.n - 1:
                    removeLen = len(sentence[i - self.n + 1])
                    cur_context = cur_context[removeLen:]

        for key in count_n.keys():
            if key[1] not in self.prob_dict:
                self.prob_dict[key[1]] = dict()
            self.prob_dict[key[1]][key[0]] = round(count_n[key] / count_n_1[key[1]], 3)
            if self.prob_dict[key[1]][key[0]] == 0.0:
                self.prob_dict[key[1]][key[0]] = 0.0001
            # if(self.prob_dict[key[1]][key[0]] == 0):
            #     print(count_n[key],count_n_1[key[1]])
            self.min_prob = min(self.min_prob, self.prob_dict[key[1]][key[0]])
        # print(self.min_prob)
        self.min_prob = log10(self.min_prob)

        # print("i:",count_n[("","of")])
        # print(count_n_1["i"])
        # print(self.prob_dict)

    # Get the probablility of a word occuring given a certain context
    def get_prob(self, context, cur_word):
        if context not in self.prob_dict or cur_word not in self.prob_dict[context]:
            # print("in:",cur_word)
            return log10(0.001) + self.min_prob
        # print(self.prob_dict[context][cur_word])
        return log10(self.prob_dict[context][cur_word])

    # Get probability with smoothing
    def get_s_prob(self, context, cur_word):
        if context not in self.prob_dict or cur_word not in self.prob_dict[context]:
            return log10(0.6 * (self.dictionary[cur_word] / self.cnt))
        return log10(
            0.6 * self.prob_dict[context][cur_word]
            - 0.00001
            + 0.4 * (self.dictionary[cur_word] / self.cnt)
        )

    def predict(self, context, cur_word):
        # Bạn có thể sử dụng hàm get_prob hoặc get_s_prob tùy thuộc vào việc sử dụng smoothing hay không
        return exp(self.get_s_prob(context, cur_word))

# Quick test
two_gram = N_gram(2)

two_gram.train(
    [["the", "sky", "is", "blue", "with", "the", "sky", "and", "is", "blue", "cool"]]
)

print(str(two_gram.predict("sky", "blue")) + " " + str(two_gram.predict("the", "blue")))

0.45560905165648535 0.45560905165648535


In [4]:
def edit_distance(s1,s2):
    dp = [[0 for j in range(len(s2)+1)] for i in range(len(s1)+1)]
    for i in range(1, len(s2)+1):
        dp[0][i] = i
    
    for i in range(1, len(s1)+1):
        dp[i][0] = i

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            dp[i][j] = min(min(dp[i-1][j],dp[i][j-1])+1, dp[i-1][j-1] + (1 if s1[i-1] != s2[j-1] else 0))
    
    return dp[len(s1)][len(s2)]

# Functions from the nlp tutorial notebook
def edits1(word):
  """All edits that are one edit away from 'word'"""
  letters = 'abcdefghijklmnopqrstuvwxyz'
  splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
 
  deletes = [L + R[1:] for L, R in splits if R ]
  transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
  replaces = [L + c + R[1:] for L, R in splits if R for c in letters ]
  inserts = [L + c + R for L, R in splits for c in letters]
  return set(deletes + transposes + replaces + inserts)

def edits2(word):
  """All the edits are two edits away from word"""
  return set((e2 for e1 in edits1(word) for e2 in edits1(e1)))

def edits3(word):
  """All the edits are two edits away from word"""
  return set((e2 for e1 in edits2(word) for e2 in edits1(e1)))

def editsk(word, k):
    e = edits1(word)
    prev = e
    for i in range(1,k):
        temp = set(e2 for e1 in prev for e2 in edits1(e1))
        e.update(temp)
        prev = temp
    e.discard(word)
    return e

# print(edit_distance("all", "ball"))
# print(editsk("tall",3) == edits2("tall").union(edits1("tall"), edits3("tall")))

In [25]:
# Spelling correction class
class Spelling_Corrector(N_gram):
    def __init__(self, n_gram=1):
        super().__init__(n_gram)

    # Generate error word within 2 edit distance of the original word
    def generate_error_word(self, word, edit_distance = 2):
        possible_errors = editsk(word, edit_distance)
        # print(possible_errors)
        non_word_errors = []
        real_word_errors = []
        for error in possible_errors:
            if error in self.dictionary:
                real_word_errors.append(error)
            else:
                non_word_errors.append(error)
        # print(real_word_errors)
        c = random.randint(0,1)
        # c = 1
        if c or len(real_word_errors)==0:
            return random.choice(non_word_errors)
        return random.choice(real_word_errors)
        
    # Generate errors in a given set of sentence 
    def generate_errors(self, data, edit_distance = 2, errors_per_sentence = 1):
        print("Generating Errors")
        cnt = 0
        for sentence in data:
            indices = random.choices(range(len(sentence)), k = errors_per_sentence)
            for i in indices:
                # print("Before: ",sentence[i])
                sentence[i] = self.generate_error_word(sentence[i],edit_distance)
                # print("After: ",sentence[i])
            if cnt%100 == 0:
                print(cnt, "completed")
            cnt += 1

        print("Errors Generated")
        return data


    # Check if there is an error present a given part of the sentence
    def is_error(self, context, word):
        if self.get_prob(context, word) < self.min_prob:
            return True

        return False
                
    # Find the correction for a word            
    def get_corrected_word(self, context, word, removeLen=inf, next_word=""):
        if removeLen == inf:
            removeLen = len(context)
        if context not in self.prob_dict:
            return word
        
        corrected_word = word
        # print(word)
        next_context = context[removeLen:]
        max_prob = self.get_prob(context,word)+self.get_prob(next_context+word, next_word)
        # if(word == "sure"):
        #     print(max_prob, "c",context,"nc", next_context,"nw", next_word)
        for candidate_word in self.dictionary:
            # if(candidate_word == "of"):
                # print(edit_distance(candidate_word, word) <= 2)
            #     print("maybe:",candidate_word)
            if (edit_distance(candidate_word, word) <= 2):
                # print("possible:", word, candidate_word)
                if self.get_prob(context, candidate_word) + self.get_prob(next_context+candidate_word, next_word) > max_prob:
                    max_prob = self.get_prob(context, candidate_word) + self.get_prob(next_context+candidate_word, next_word)
                    corrected_word = candidate_word
            # print()
        
        return corrected_word

    # Detect and correct all errors in a set of sentences
    def find_and_correct_errors(self, data_with_errors, max_to_correct = 1):
        print("Correction Started")
        corrected_data = data_with_errors
        cnt = 0
        for j in range(len(corrected_data)):
            cur_context = ''
            corrected_cnt = 0
            minp = inf
            mini = -1
            for i in range(len(corrected_data[j])):
                if corrected_cnt == max_to_correct:
                    break
                # if(self.is_error(cur_context, corrected_data[j][i])):
                next_word = ""
                if i+1 < len(corrected_data[j]):
                    next_word = corrected_data[j][i+1]
                cur_prob = self.get_prob(cur_context, corrected_data[j][i])
                if(corrected_data[j][i] not in self.dictionary):
                    # print("err:",corrected_data[j][i])
                    removeLen = 0
                    if i >= self.n-1:
                        removeLen = len(corrected_data[j][i-self.n+1])
                    
                    # print("word:",cur_context, corrected_data[j][i], removeLen, next_word)
                    bef = corrected_data[j][i]
                    corrected_data[j][i] = self.get_corrected_word(cur_context, corrected_data[j][i], removeLen, next_word)
                    if bef != corrected_data[j][i]:
                        corrected_cnt+=1
                    
                cur_context+=corrected_data[j][i]
                
                if i >= self.n-1:
                    removeLen = len(corrected_data[j][i-self.n+1])
                    cur_context = cur_context[removeLen:]
                
                next_prob = self.get_prob(cur_context, next_word)
                # print("c",cur_context, "w", corrected_data[j][i], "nw", next_word)
                if cur_prob + next_prob < minp:
                    minp = cur_prob + next_prob
                    mini = i

            cur_context = ""
            # print(minp, corrected_data[j][mini])
            # print("core",corrected_cnt)

            for i in range(len(corrected_data[j])):
                if corrected_cnt == max_to_correct:
                    break
                # print(cur_context, corrected_data[j][i])
                if(i == mini):
                    # print("err:",corrected_data[j][i])
                    removeLen = 0
                    if i >= self.n-1:
                        removeLen = len(corrected_data[j][i-self.n+1])
                    next_word = ""
                    if i+1 < len(corrected_data[j]):
                        next_word = corrected_data[j][i+1]
                    # print("word:",cur_context, corrected_data[j][i], removeLen, next_word)
                    corrected_data[j][i] = self.get_corrected_word(cur_context, corrected_data[j][i], removeLen, next_word)
                    corrected_cnt+=1
                cur_context+=corrected_data[j][i]
                if i >= self.n-1:
                    removeLen = len(corrected_data[j][i-self.n+1])
                    cur_context = cur_context[removeLen:]
            if cnt % 100 == 0:
                print(cnt, "Completed")

            cnt += 1 
            
            # print()
        print("Correction Done")
        return corrected_data


    
        
#Quick Test
# corrector = Spelling_Corrector(2)
# corrector.train([["the", "sky", "is", "blue", "with", "the", "sky", "and", "is", "blue"], ["there", "is", "the", "sky", "blue", "in", "color"]])
# test = [["there", "is", "the", "sky"], ["the", "sky", "is", "blue"]]
# data_with_errors = test.copy()
# data_with_errors = corrector.generate_errors(data_with_errors, 2)
# print(data_with_errors)
# corrected_data = corrector.find_and_correct_errors(data_with_errors)
# print(corrected_data)

In [7]:
class Problem:
    def __init__(self, data, solve):
        self.data = data
        self.solve = solve

    
    # Function to preprocess the data for a given problem
    def preprocess(self):
        # Remove punctuations that come in middle of sentences
        punctuations = set((',', '--', '-', ':', ';', '``', "''", "'", '"', "`", '(', ')', '[', ']', ',"', 's','--(', ')--', '--[', ']--', '--"', "--`", "--'", "CHAPTER", ",--", ":--"))
        processed_data = [self.data[i].lower().strip('_') for i in range(len(self.data)) if ((self.data[i] not in punctuations) and (i>0 and self.data[i-1]!="CHAPTER"))]
        processed_data = [processed_data[i] for i in range(len(processed_data)) if ((processed_data[i]!='.') or (i>0 and processed_data[i]=='.' and processed_data[i-1] not in set(["mr", "mrs", "dr"])))]
        processed_data = [processed_data[i] for i in range(len(processed_data)) if ((len(processed_data[i]) > 1 and processed_data[i] in english_words_lower_set) or (processed_data[i] in set(['0','1','2','3','4','5','6','7','8','9','.','a','i','o']) ))]
        # processed_data = [processed_data[i] for i in range(len(processed_data)) if processed_data[i] in english_words_lower_set]
        l = [len(x) for x in processed_data if len(x)!=1]
        print("Avg word len:", sum(l)/len(l))

        # Split data into list of sentences using the puntuation marks coming at the end of the sentence
        breakpoints = [i for i in range(len(processed_data)) if processed_data[i] in set([".","?","!",".--", '.--"',"?--", '?--"',"!--", '!--"', ";--", '."', '?"', '!"'])]
        sentences = []
        prev = 0
        for i in range(len(breakpoints)):
            sentences.append(processed_data[prev:breakpoints[i]])
            prev = breakpoints[i]+1

        processed_data = [s for s in sentences if len(s) > 1]
        
        self.processed_data = processed_data

    # print(sentences)

    # Function to split data into train and test sets
    def train_test_split(self, ratio = 0.8):
        random.shuffle(self.processed_data)
        split_point = ceil(ratio*len(self.processed_data))
        self.train_data, self.test_data = self.processed_data[:split_point],self.processed_data[split_point:]
    
    # Use the solve function to solve the problem
    def answer(self, ratio = 1):
        self.solve(self.train_data, self.test_data[:ceil(ratio*len(self.test_data))])

    
#Solve function for P1
def solvep1(train_data, test_data):
    corrector = Spelling_Corrector(2)
    corrector.train(train_data)
    
    data_with_errors = copy.deepcopy(test_data)
    data_with_errors = corrector.generate_errors(data_with_errors, 2, 1)
    print("error:", data_with_errors)
    corrected_data = copy.deepcopy(data_with_errors)
    corrected_data = corrector.find_and_correct_errors(corrected_data, 1)
    print("correct:", corrected_data)
    correct_cnt = 0
    for i in range(len(corrected_data)):
        print("actual:", test_data[i])
        print("error:", data_with_errors[i])
        print("corrected:", corrected_data[i])
        if test_data[i] == corrected_data[i]:
            correct_cnt+=1

    print("Accuracy: ", correct_cnt,"/",len(corrected_data), "=", correct_cnt/len(corrected_data))

p1 = Problem(list(gutenberg.words('austen-emma.txt')),solvep1)
p1.preprocess()
print(len(p1.processed_data))
p1.train_test_split(0.8)
print(len(p1.train_data), len(p1.test_data))
p1.answer()

Avg word len: 3.8255566136126604
5027
4022 1005
Generating Errors
0 completed
100 completed
200 completed
300 completed
400 completed
500 completed
600 completed
700 completed
800 completed
900 completed
1000 completed
Errors Generated
error: [['for', 'two', 'or', 'three', 'i', 'shall', 'where', 'i', 'am', 'and', 'as', 'i', 'am', 'and', 'i', 'am', 'quite', 'serious', 'too', 'i', 'assure', 'you', 'mrs', 'elton', 'in', 'to', 'be', 'always', 'on', 'the', 'watch', 'and', 'he', 'to', 'watch', 'also', 'that', 'nothing', 'may', 'pass', 'us', 'in', 'this', 'style', 'she', 'ran', 'on', 'never', 'by', 'any', 'thing', 'till', 'mr', 'came', 'into', 'the', 'room', 'her', 'vanity', 'had', 'then', 'a', 'change', 'of', 'object', 'and', 'emma', 'heard', 'her', 'in', 'the', 'same', 'half', 'whisper', 'to', 'jane', 'here', 'this', 'dear', 'old', 'beau', 'of', 'mine', 'i', 'protest', 'only', 'think', 'of', 'his', 'gallantry', 'in', 'away', 'before', 'the', 'other', 'men', 'what', 'a', 'dear', 'creature', 